
# <font color="#1d479b">Sommaire</font>
1. [Data Preprocessing](#section_1)     
    1.1. [Préparation des données](#section_1_1)  
    1.2. [Encodage et Standardisation](#section_1_2)     
    1.3. [Separation Train set et Test set](#section_1_3)     

2. [Predictions](#section_2)     
    1.1. [Linear Regression](#section_2_1)  
    1.2. [Random Forest](#section_2_2)     
    1.3. [SVR](#section_2_3)   
    1.3. [XGBoost](#section_2_3) 

In [ ]:
pip install plotly --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 7.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: plotly
    Found existing installation: plotly 5.6.0
    Uninstalling plotly-5.6.0:
      Successfully uninstalled plotly-5.6.0


In [8]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

# Load the data for the country
df = pd.read_csv("departements-region.csv")

# Fit the k-means clustering model to the data
model = KMeans(n_clusters=3)
model.fit(df[["population"]])

# Add the cluster assignments to the dataframe
df["cluster"] = model.labels_

# Create the Plotly Express choropleth map
fig = px.choropleth(df, locations="code", color="cluster", title="Population Clusters by Region in France",
                   color_continuous_scale=px.colors.sequential.Plasma,
                   scope="europe", text="region",
                   projection="mercator")

# Show the map
fig.show()


KeyError: "None of [Index(['population'], dtype='object')] are in the [columns]"

In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

# Load the data for France
df = pd.read_csv("departements-region.csv")

# Create the Plotly Express map
fig = px.choropleth(df, locations="code", color="population", title="Population by Region in France",
                   color_continuous_scale=px.colors.sequential.Plasma,
                   geo="geo", text="region",
                   projection="mercator")

# Create the Dash application
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True)


TypeError: choropleth() got an unexpected keyword argument 'geo'

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, confusion_matrix

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline

plt.style.use('ggplot')
sns.set_style("whitegrid")
sns.color_palette("crest", as_cmap=True)
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)

In [7]:
df = pd.read_csv("data_seattle.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,ZipCode,YearBuilt,ENERGYSTARScore,SiteEnergyUse,TotalGHGEmissions,GFAPerBuilding,GFAPerFloor,GFABuildingRate,BuildingType_Campus,BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Address_Avenue,Address_Others,Address_Street,Address_Way,Neighborhood_Ballard,Neighborhood_Central,Neighborhood_Delridge,Neighborhood_Downtown,Neighborhood_East,Neighborhood_Greater Duwamish,Neighborhood_Lake Union,Neighborhood_Magnolia / Queen Anne,Neighborhood_North,Neighborhood_Northeast,Neighborhood_Northwest,Neighborhood_Southeast,Neighborhood_Southwest,LargestPropertyUseType_Bank Branch,LargestPropertyUseType_Courthouse,LargestPropertyUseType_Data Center,LargestPropertyUseType_Distribution Center,LargestPropertyUseType_Financial Office,LargestPropertyUseType_Hospital (General Medical & Surgical),LargestPropertyUseType_Hotel,LargestPropertyUseType_K-12 School,LargestPropertyUseType_Medical Office,LargestPropertyUseType_Multifamily Housing,LargestPropertyUseType_Non-Refrigerated Warehouse,LargestPropertyUseType_Office,LargestPropertyUseType_Other,LargestPropertyUseType_Parking,LargestPropertyUseType_Refrigerated Warehouse,LargestPropertyUseType_Residence Hall/Dormitory,LargestPropertyUseType_Retail Store,LargestPropertyUseType_Senior Care Community,LargestPropertyUseType_Supermarket/Grocery Store,LargestPropertyUseType_Wholesale Club/Supercenter,LargestPropertyUseType_Worship Facility
0,98101.0,1927,60.0,7226362.5,249.98,88434.0,7369.500,1.00000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,98101.0,1996,61.0,8387933.0,295.86,103566.0,9415.091,0.85455,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,98101.0,1969,43.0,72587024.0,2089.28,956110.0,23319.756,0.79425,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,98101.0,1926,56.0,6794584.0,286.43,61320.0,6132.000,1.00000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,98121.0,1980,75.0,14172606.0,505.01,175580.0,9754.444,0.64688,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 1.  DATASETS

- **Données Avec ENERGYSTARScore**

In [11]:
df_with_ENSTScore = df.copy()
df_with_ENSTScore.head()

,ZipCode,YearBuilt,ENERGYSTARScore,SiteEnergyUse,TotalGHGEmissions,GFAPerBuilding,GFAPerFloor,GFABuildingRate,BuildingType_Campus,BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Address_Avenue,Address_Others,Address_Street,Address_Way,Neighborhood_Ballard,Neighborhood_Central,Neighborhood_Delridge,Neighborhood_Downtown,Neighborhood_East,Neighborhood_Greater Duwamish,Neighborhood_Lake Union,Neighborhood_Magnolia / Queen Anne,Neighborhood_North,Neighborhood_Northeast,Neighborhood_Northwest,Neighborhood_Southeast,Neighborhood_Southwest,LargestPropertyUseType_Bank Branch,LargestPropertyUseType_Courthouse,LargestPropertyUseType_Data Center,LargestPropertyUseType_Distribution Center,LargestPropertyUseType_Financial Office,LargestPropertyUseType_Hospital (General Medical & Surgical),LargestPropertyUseType_Hotel,LargestPropertyUseType_K-12 School,LargestPropertyUseType_Medical Office,LargestPropertyUseType_Multifamily Housing,LargestPropertyUseType_Non-Refrigerated Warehouse,LargestPropertyUseType_Office,LargestPropertyUseType_Other,LargestPropertyUseType_Parking,LargestPropertyUseType_Refrigerated Warehouse,LargestPropertyUseType_Residence Hall/Dormitory,LargestPropertyUseType_Retail Store,LargestPropertyUseType_Senior Care Community,LargestPropertyUseType_Supermarket/Grocery Store,LargestPropertyUseType_Wholesale Club/Supercenter,LargestPropertyUseType_Worship Facility
0,98101.0,1927,60.0,7226362.5,249.98,88434.0,7369.500,1.00000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,98101.0,1996,61.0,8387933.0,295.86,103566.0,9415.091,0.85455,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,98101.0,1969,43.0,72587024.0,2089.28,956110.0,23319.756,0.79425,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,98101.0,1926,56.0,6794584.0,286.43,61320.0,6132.000,1.00000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,98121.0,1980,75.0,14172606.0,505.01,175580.0,9754.444,0.64688,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- **Données sans ENERGYSTARScore**

In [9]:
# DATASET WITH ENERGYSTARScore 
df_No_ENSTScore = df.drop(columns=["ENERGYSTARScore"])

In [10]:
df_No_ENSTScore.head()

,ZipCode,YearBuilt,SiteEnergyUse,TotalGHGEmissions,GFAPerBuilding,GFAPerFloor,GFABuildingRate,BuildingType_Campus,BuildingType_NonResidential,BuildingType_Nonresidential COS,BuildingType_SPS-District K-12,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Address_Avenue,Address_Others,Address_Street,Address_Way,Neighborhood_Ballard,Neighborhood_Central,Neighborhood_Delridge,Neighborhood_Downtown,Neighborhood_East,Neighborhood_Greater Duwamish,Neighborhood_Lake Union,Neighborhood_Magnolia / Queen Anne,Neighborhood_North,Neighborhood_Northeast,Neighborhood_Northwest,Neighborhood_Southeast,Neighborhood_Southwest,LargestPropertyUseType_Bank Branch,LargestPropertyUseType_Courthouse,LargestPropertyUseType_Data Center,LargestPropertyUseType_Distribution Center,LargestPropertyUseType_Financial Office,LargestPropertyUseType_Hospital (General Medical & Surgical),LargestPropertyUseType_Hotel,LargestPropertyUseType_K-12 School,LargestPropertyUseType_Medical Office,LargestPropertyUseType_Multifamily Housing,LargestPropertyUseType_Non-Refrigerated Warehouse,LargestPropertyUseType_Office,LargestPropertyUseType_Other,LargestPropertyUseType_Parking,LargestPropertyUseType_Refrigerated Warehouse,LargestPropertyUseType_Residence Hall/Dormitory,LargestPropertyUseType_Retail Store,LargestPropertyUseType_Senior Care Community,LargestPropertyUseType_Supermarket/Grocery Store,LargestPropertyUseType_Wholesale Club/Supercenter,LargestPropertyUseType_Worship Facility
0,98101.0,1927,7226362.5,249.98,88434.0,7369.500,1.00000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,98101.0,1996,8387933.0,295.86,103566.0,9415.091,0.85455,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,98101.0,1969,72587024.0,2089.28,956110.0,23319.756,0.79425,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,98101.0,1926,6794584.0,286.43,61320.0,6132.000,1.00000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,98121.0,1980,14172606.0,505.01,175580.0,9754.444,0.64688,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
# Definition of target and features variables

targets = ['SiteEnergyUse', 'TotalGHGEmissions']

target1 = ['SiteEnergyUse']
target2 = ['TotalGHGEmissions']

X = df_No_ENSTScore.iloc[:, ~df_No_ENSTScore.columns.isin(targets)]
y = df_No_ENSTScore.loc[:, target1]

In [51]:
print(X.shape)
print(y.shape)

(995, 64)
(995, 1)


## 1. LINEAR REGRESSION

In [52]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [53]:
# Fit a linear regression model
model_lr = LinearRegression()
model_lr.fit(X, y)

LinearRegression()

In [54]:
# Perform cross-validation
cv_results = cross_val_score(model_lr, X, y, cv=5, scoring='neg_mean_squared_error')


In [55]:
# Calculate the RMSE, MSE, MAE, and R^2
rmse = np.sqrt(-cv_results).mean()
mse = -cv_results.mean()
mae = -cv_results.mean()
r2 = model_lr.score(X, y)


In [56]:
print("Root Mean Absolute Error:", rmse)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R squared:", r2)

Root Mean Absolute Error: 1.3921455986708818e+20
Mean Squared Error: 3.64835856054096e+40
Mean Absolute Error: 3.64835856054096e+40
R squared: 0.5710408999441738


# 2. Random Forest

In [57]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [58]:
# Split the data into features (X) and target (y)
y = df_No_ENSTScore.loc[:, target2]


In [59]:
# Fit a Random Forest Regression model
model_rf = RandomForestRegressor(n_estimators=100)
model_rf.fit(X, y)


RandomForestRegressor()

In [60]:
# Perform cross-validation
cv_results = cross_val_score(model_rf, X, y, cv=5, scoring='neg_mean_squared_error')


In [61]:
# Calculate the RMSE, MSE, MAE, and R^2
rmse = np.sqrt(-cv_results).mean()
mse = -cv_results.mean()
mae = -cv_results.mean()
r2 = model_rf.score(X, y)

print("Root Mean Absolute Error:", rmse)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R squared:", r2)


Root Mean Absolute Error: 651.1680190137865
Mean Squared Error: 561411.808453684
Mean Absolute Error: 561411.808453684
R squared: 0.9221872202017269


# 5. IMPACT OF THE ENERGYSTARSCore